In [1]:
# Import basic data science packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Sequential Keras model for adding layers
from tensorflow.keras.models import Sequential

# Import different types of CNN layers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix

In [52]:
batch_size = 128
epochs = 10

In [53]:
# Rescale images so pixel values are between [0, 1]
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1/255)

In [54]:
# Create iterator for train and test sets (default batch size is 32)
train_generator = train_datagen.flow_from_directory('data/train', 
                                                    class_mode='binary',
                                                    target_size=(96,96))

test_generator = test_datagen.flow_from_directory('data/test', 
                                                 class_mode='binary', 
                                                 target_size=(96,96))

Found 176021 images belonging to 2 classes.
Found 44004 images belonging to 2 classes.


In [35]:
# Simple CNN model architecture, with Max Pooling for dimensionality reduction and dropout layer to reduce overfiting
CNN_model = Sequential()

# Input shape is 96x96 px and 3 colour channels
CNN_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape = (96, 96, 3)))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25))

CNN_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
CNN_model.add(MaxPooling2D(pool_size=(2, 2)))
CNN_model.add(Dropout(0.25))

# Flatten the output of the convolutional layers
CNN_model.add(Flatten())

# Add dense layers
CNN_model.add(Dense(64, activation='relu'))
CNN_model.add(Dense(64, activation='relu'))

# Add final dense layer for output
CNN_model.add(Dense(2, activation='softmax')) 
# CNN_model.add(Dense(1, activation='sigmoid'))

# Print out a summary of the network
CNN_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 36864)            

In [55]:
# Compile the model with the desired loss function, optimizer, and metric(s) to track
CNN_model.compile(loss='binary_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])

In [56]:
CNN_model.fit_generator(train_generator, 
              steps_per_epoch=176021//batch_size, 
              epochs=epochs, 
              verbose=1)

Epoch 1/10
1375/1375 [==============================] - 203s 148ms/step - loss: 0.6951 - accuracy: 0.4522
Epoch 2/10
1375/1375 [==============================] - 204s 148ms/step - loss: 0.6932 - accuracy: 0.4732
Epoch 3/10
1375/1375 [==============================] - 202s 147ms/step - loss: 0.6934 - accuracy: 0.4992
Epoch 4/10
1375/1375 [==============================] - 327s 238ms/step - loss: 0.6932 - accuracy: 0.4666
Epoch 5/10
1375/1375 [==============================] - 205s 149ms/step - loss: 0.6932 - accuracy: 0.4961
Epoch 6/10
1375/1375 [==============================] - 230s 168ms/step - loss: 0.6932 - accuracy: 0.5204
Epoch 7/10
1375/1375 [==============================] - 208s 152ms/step - loss: 0.6932 - accuracy: 0.5047
Epoch 8/10
 219/1375 [===>..........................] - ETA: 2:57 - loss: 0.6931 - accuracy: 0.5087

KeyboardInterrupt: 

In [ ]:
# Evaluate the model's performance on the test data
score = CNN_model.evaluate_generator(test_generator, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [47]:
# Calculate the predicted labels for each test image.
predict_probas = CNN_model.predict(test_generator)

KeyboardInterrupt: 

In [ ]:
y_predict = np.argmax(predict_probas, axis=1)

# Create the confusion matrix using sklearn 
conf_mat = confusion_matrix(y_test, y_predict)

# Since we have many images, it is helpful to show our 
# results as fractions of the total number of images 
# for each class.
normalized_conf_mat = conf_mat / conf_mat.sum(axis=1)

plt.figure(figsize = (9,7))
sns.heatmap(normalized_conf_mat,
            annot=True,
            cbar=False,
            cmap="rocket_r",
            linewidths=1
           )
plt.title('Confusion Matrix',size = 25,y=1.01)
plt.xlabel("Predicted Label", size = 20)
plt.ylabel("True Label", size = 20)
plt.show()

In [ ]:
incorrect_photos = (y_test != y_predict)

num_images = 20
columns = 5
rows = 4

fig, ax = plt.subplots(nrows=rows, ncols=columns, figsize=(10, 10))

ax = ax.flatten()

for i, image in enumerate(X_test[incorrect_photos][:num_images]):
    ax[i].imshow(image, cmap='gray')

    # Get predicted label
    prediction = y_predict[incorrect_photos][i]
    
    # Get actual label
    actual = y_test[incorrect_photos][i]
    
    # Set the title of the subplot
    ax[i].set_title(f"Predicted: {prediction}\n Actual: {actual}")
    
    # Hide the x and y ticks to make 
    ax[i].set_xticks([]) 
    ax[i].set_yticks([])
    
fig.tight_layout()